#### BDP Assignment 4

#### ID: 202318002
#### Name: Bhavik Manwani

In [1]:
# installing pyspark
# !pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=7456ac03b01f8f73e79388cb375abeca00f0c7603bb29c3f46e839ab9748b2f2
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [16]:
# importing libraries

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import numpy as np

In [3]:
# creating spark session using app name and local master

spark = SparkSession.builder.appName("RDD").master("local").getOrCreate()
spark

##### RDD Task 1
Generate 100 random numbers in range 0 to 10 using numpy randint function with the seed set to 10. Create a RDD using the parallelize function using data generated in previous step. Calculate the frequency of each number (0 - 10) using appropriate function of RDD.

In [53]:

np.random.seed(10)

# generating random array from 0 to 10 of size 100
arr = np.random.randint(0, 10, 100)

# parellelizing arr using spark context parellelize
num_rdd = spark.sparkContext.parallelize(arr)

# mapping each number to frequency 1
num_rdd = num_rdd.map(lambda x: (x,1))

# counting frequency of each number
num_count = num_rdd.reduceByKey(lambda a,b: a+b)

# sorting numbers by value
num_count.sortByKey().collect()

[(0, 14),
 (1, 14),
 (2, 9),
 (3, 6),
 (4, 8),
 (5, 5),
 (6, 11),
 (7, 5),
 (8, 14),
 (9, 14)]

##### RDD Task 2
In this task you will calculate the frequency of each word in text8 dataset mentioned above. Create a RDD using the text8 dataset.
Use appropriate functions of the RDD to get the word frequencies. Filter the RDD using appropriate function to get the frequencies of words containing the letter ’a’.

In [61]:
# reading text file through spark context

rdd = spark.sparkContext.textFile('text8') # RDD created
rdd

text8 MapPartitionsRDD[186] at textFile at NativeMethodAccessorImpl.java:0

In [62]:

# splitting file by whitespaces to fetch words by rdd flatMap
rdd_split = rdd.flatMap(lambda x: x.split(' '))

# filtering rdd object for words containing 'a'
rdd_filtered = rdd_split.filter(lambda x : 'a' in x)

rdd_filtered.collect()[:10]

['anarchism',
 'originated',
 'as',
 'a',
 'abuse',
 'against',
 'early',
 'class',
 'radicals',
 'and']

In [63]:

# assigning each word single frequency
rdd_filtered = rdd_filtered.map(lambda x: (x,1))

# counting frequency of each word
word_counts = rdd_filtered.reduceByKey(lambda a,b: a+b)

# sortign words by value
word_counts = word_counts.sortByKey()
word_counts.take(5)

[('a', 325873),
 ('aa', 276),
 ('aaa', 57),
 ('aaaa', 7),
 ('aaaaaacceglllnorst', 1)]

##### DataFrame Task:
Create a Spark dataframe using the iris json data mentioned above. Calculate Pearson Correlation between the columns petalLength and petalWidth using the appropriate dataframe API.
Show the columns sepalLength, sepalWidth and species for the rows of data that has petalLength greater than or equal to 1.4 using the appropriate dataframe API.

In [72]:

# reading json file
df = spark.read.json("iris.json")
df.show()

# Here df shows '_corrupt_record' column which is to be removed


+---------------+-----------+----------+-----------+----------+-------+
|_corrupt_record|petalLength|petalWidth|sepalLength|sepalWidth|species|
+---------------+-----------+----------+-----------+----------+-------+
|              [|       NULL|      NULL|       NULL|      NULL|   NULL|
|           NULL|        1.4|       0.2|        5.1|       3.5| setosa|
|           NULL|        1.4|       0.2|        4.9|       3.0| setosa|
|           NULL|        1.3|       0.2|        4.7|       3.2| setosa|
|           NULL|        1.5|       0.2|        4.6|       3.1| setosa|
|           NULL|        1.4|       0.2|        5.0|       3.6| setosa|
|           NULL|        1.7|       0.4|        5.4|       3.9| setosa|
|           NULL|        1.4|       0.3|        4.6|       3.4| setosa|
|           NULL|        1.5|       0.2|        5.0|       3.4| setosa|
|           NULL|        1.4|       0.2|        4.4|       2.9| setosa|
|           NULL|        1.5|       0.1|        4.9|       3.1| 

In [77]:

# discarding column with index 1 from df
df1 = df.select(df.columns[1:])

# calculating pearson correlation between columns
c = df1.corr('petalLength', 'petalWidth', method='pearson')

print(f'\nCorrelation: {c} \n')



Correlation: 0.9626417223780231 



In [74]:
# filtering records with given condition
df1.filter((df1.petalLength >= 1.4)).show()


+-----------+----------+-----------+----------+-------+
|petalLength|petalWidth|sepalLength|sepalWidth|species|
+-----------+----------+-----------+----------+-------+
|        1.4|       0.2|        5.1|       3.5| setosa|
|        1.4|       0.2|        4.9|       3.0| setosa|
|        1.5|       0.2|        4.6|       3.1| setosa|
|        1.4|       0.2|        5.0|       3.6| setosa|
|        1.7|       0.4|        5.4|       3.9| setosa|
|        1.4|       0.3|        4.6|       3.4| setosa|
|        1.5|       0.2|        5.0|       3.4| setosa|
|        1.4|       0.2|        4.4|       2.9| setosa|
|        1.5|       0.1|        4.9|       3.1| setosa|
|        1.5|       0.2|        5.4|       3.7| setosa|
|        1.6|       0.2|        4.8|       3.4| setosa|
|        1.4|       0.1|        4.8|       3.0| setosa|
|        1.5|       0.4|        5.7|       4.4| setosa|
|        1.4|       0.3|        5.1|       3.5| setosa|
|        1.7|       0.3|        5.7|       3.8| 